In [40]:
import numpy as np
from sklearn.preprocessing import normalize
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import os, sys
sys.path.append(os.getcwd())
import filter_eeg as fe

In [41]:
df = pd.read_pickle("../data/data_final.pkl")
# df = df[df.is_corrupt == 0] #remove corrupt readings
# df = df.assign(win = lambda x: x.episode_type.isin([0,3])*1) #create win column. 1 if won, 0 if lost.
# df = df.assign(left = lambda x: x.episode_type.isin([0,2])*1) #create left column. 1 if the reward is on the left, 0 if it's on the right.
labels = ["episode_type","trial_type","trial_num",
          "episode_num","is_corrupt","action_index",
          "trimmed_data","left","subject_num","win"]
# df = df.drop(labels, axis=1)
df = df.reset_index(drop=True)

### resample at lower hertz

In [42]:
resamp = 100 # 100Hz
resamp = int(512/resamp)

eeg_df, labels_df = df[list(range(1880))], df[labels]
eeg_ = pd.DataFrame()
for r in range(eeg.shape[0]):
    row = eeg.iloc[r]
    row_ = []
    for c in range(resamp, eeg.shape[1], resamp):
        row_.append(np.mean(row[c-resamp: c]))
    eeg_ = eeg_.append([row_],ignore_index=True)

df = eeg_.join(labels_df)

df.describe()

,0,1,2,3,4,5,6,7,8,9,...,episode_type,trial_type,trial_num,episode_num,is_corrupt,action_index,trimmed_data,left,subject_num,win
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,...,1161.000000,1161.000000,1161.000000,1161.000000,1161.0,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,53.658915,54.688544,54.459087,54.496985,51.034281,51.154522,50.852713,53.579500,56.256331,59.561929,...,1.551249,0.485788,19.989664,14.476314,0.0,1025.596038,34.442722,0.484927,5.433247,0.616710
std,82.088600,92.533246,89.430760,84.986394,78.625794,73.616756,71.184671,71.539571,73.862749,77.954870,...,1.217403,0.500013,11.250589,8.650274,0.0,8.674554,8.294080,0.499988,2.878139,0.486398
min,-621.000000,-1182.600000,-1100.600000,-890.000000,-511.200000,-387.400000,-391.400000,-397.600000,-386.200000,-409.400000,...,0.000000,0.000000,1.000000,0.000000,0.0,987.000000,1.000000,0.000000,1.000000,0.000000
25%,20.600000,24.000000,30.200000,33.600000,31.400000,30.600000,31.800000,31.000000,33.800000,36.600000,...,0.000000,0.000000,10.000000,7.000000,0.0,1024.000000,34.000000,0.000000,3.000000,0.000000
50%,56.400000,57.400000,57.400000,58.400000,54.200000,55.400000,55.400000,58.200000,57.600000,60.000000,...,2.000000,0.000000,20.000000,14.000000,0.0,1025.000000,34.000000,0.000000,5.000000,1.000000
75%,84.200000,85.800000,83.200000,83.600000,80.600000,81.200000,80.000000,82.800000,83.600000,82.600000,...,3.000000,1.000000,30.000000,22.000000,0.0,1025.000000,38.000000,1.000000,8.000000,1.000000
max,681.200000,690.000000,675.200000,517.800000,434.400000,561.000000,648.200000,634.000000,510.800000,536.600000,...,3.000000,1.000000,39.000000,29.000000,0.0,1062.000000,76.000000,1.000000,10.000000,1.000000


### break original df into one df per subject and put in list

In [43]:
num_subjects = 10
dfs = []

#calculate trim extremes
num_readings =  df.shape[1]
s,e = int(num_readings*.05), int(num_readings*.95)

for i in range(1,num_subjects+1):
#     sbj_df = df[df.subject_num == i]
#     sbj_df_eeg = sbj_df.drop(labels, axis=1)
#     sbj_df_labels = sbj_df[labels]
    
    sbj_w = df[(df.subject_num == i) & (df.win == 1)] #df containing all wins for subject i
    sbj_w_data = sbj_w[list(range(s, e))].reset_index(drop=True)
    sbj_w_labels = sbj_w[labels].reset_index(drop=True)
#     print("\n\n\n\nsubject",i,"win df:\n",sbj_w_labels[["subject_num","win"]].describe(),"\n\n")
    
    sbj_l = df[(df.subject_num == i) & (df.win == 0)] #df containing all losses for subject i
    sbj_l_data = sbj_l[list(range(s, e))].reset_index(drop=True)
    sbj_l_labels = sbj_l[labels].reset_index(drop=True)
#     print("subject",i,"loss df:\n",sbj_l_labels[["subject_num","win"]].describe())
    
    dfs.append([[sbj_w_data, sbj_w_labels], [sbj_l_data, sbj_l_labels]])

### calculate averages within each subject for wins/losses

In [44]:
aves = []

for i in range(len(dfs)):
    av_w = np.array(dfs[i][0][0].mean()) #composite/average win raw eeg
#     norm_w = np.linalg.norm(list(av_w)) #normalize the average
#     av_w = av_w/norm_w
    av_w = normalize(av_w[:,np.newaxis], axis=0).ravel()

    
    av_l = np.array(dfs[i][1][0].mean()) #composite/avearge loss raw eeg
#     norm_l = np.linalg.norm(list(av_l)) 
#     av_l = av_l/norm_l
    av_l = normalize(av_l[:,np.newaxis], axis=0).ravel()

    
    aves.append([av_w, av_l])

### scale raw eeg value at every timestep by normalized average of wins/losses for a subject

In [45]:
for i in range(len(dfs)):
    wins = dfs[i][0][0]
    wins_ = pd.DataFrame()
    for ep in range(wins.shape[0]):
        ep_data = np.array(wins.iloc[ep])
        ep_data = list(np.multiply(ep_data, aves[i][0])) #multiply each row of raw eeg data by the normalized average for this subject/outcome
        wins_ = wins_.append([ep_data], ignore_index=True)
        
    
    losses = dfs[i][1][0]
    losses_ = pd.DataFrame()
    for ep in range(losses.shape[0]):
        ep_data = np.array(losses.iloc[ep])
        ep_data = list(np.multiply(ep_data, aves[i][1])) #multiply each row of raw eeg data by the normalized average for this subject/outcome
        losses_ = losses_.append([ep_data], ignore_index=True)
    
    dfs[i][0][0] = wins_
    dfs[i][1][0] = losses_

### zip everything back together

In [46]:
df_ = pd.DataFrame()
for i in range(len(dfs)):
    wins = dfs[i][0][0].join((dfs[i][0][1]))
    losses = dfs[i][1][0].join(dfs[i][1][1])
    subj = pd.concat([wins,losses], ignore_index = True)
    df_ = pd.concat([df_, subj], ignore_index = True)

In [47]:
df_.describe()

,0,1,2,3,4,5,6,7,8,9,...,episode_type,trial_type,trial_num,episode_num,is_corrupt,action_index,trimmed_data,left,subject_num,win
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,...,1161.000000,1161.000000,1161.000000,1161.000000,1161.0,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,3.218856,3.162438,3.032076,2.906566,2.871996,2.893409,2.963716,3.173706,3.499746,3.733398,...,1.551249,0.485788,19.989664,14.476314,0.0,1025.596038,34.442722,0.484927,5.433247,0.616710
std,4.183078,4.249418,4.160004,3.824425,3.636471,3.446742,3.508674,3.650501,3.972553,4.129333,...,1.217403,0.500013,11.250589,8.650274,0.0,8.674554,8.294080,0.499988,2.878139,0.486398
min,-35.408770,-36.062234,-33.722579,-32.042085,-32.244882,-35.319178,-33.218692,-27.933940,-21.588737,-18.657739,...,0.000000,0.000000,1.000000,0.000000,0.0,987.000000,1.000000,0.000000,1.000000,0.000000
25%,1.817760,1.689197,1.566943,1.580397,1.559179,1.667601,1.716362,1.800247,1.942850,2.177716,...,0.000000,0.000000,10.000000,7.000000,0.0,1024.000000,34.000000,0.000000,3.000000,0.000000
50%,3.157827,3.134136,2.986542,2.927393,2.839963,2.892483,2.974394,3.105543,3.570791,3.762869,...,2.000000,0.000000,20.000000,14.000000,0.0,1025.000000,34.000000,0.000000,5.000000,1.000000
75%,4.570747,4.528262,4.426292,4.301215,4.162741,4.280895,4.363857,4.758344,5.391475,5.472068,...,3.000000,1.000000,30.000000,22.000000,0.0,1025.000000,38.000000,1.000000,8.000000,1.000000
max,33.056650,33.770907,37.167198,24.965349,24.877358,24.692057,24.913644,21.628917,33.449568,45.827411,...,3.000000,1.000000,39.000000,29.000000,0.0,1062.000000,76.000000,1.000000,10.000000,1.000000


In [48]:
# df_.to_pickle("data/data_scaled.pkl")
# df_.to_pickle("data/data_processed.pkl")